In [ ]:
import glob

import numpy as np
import scipy.optimize as op

import matplotlib.pyplot as plt

import astropy.units as u

import emcee

from measure_extinction.stardata import StarData
from measure_extinction.extdata import ExtData
from measure_extinction.modeldata import ModelData
from measure_extinction.utils.fit_model import FitInfo

from measure_extinction.utils.fit_model import get_best_fit_params, get_percentile_params

from helpers import plot_data_model, get_fit_params, get_full_params

Specify the location of the model and observed data

In [ ]:
data_file_path = "/home/kgordon/STScI/AbsFlux/CalSpec/absflux_fitseds/data/faintwds/"
model_file_path = "/home/kgordon/Python/extstar_data/"

Define star specific parameters

In [ ]:
starname = "wdfs1022_30"
fstarname = f"{starname}.dat"
velocity = 0.0   # assumption
relband = "WFC3_F625W"

# parameter names
pnames = ["logT","logg", "logZ", "Av","Rv","C2","C3","C4","x0","gamma","HI_gal","HI_mw"]
pnames_fit = ["logT","logg", "Av","Rv","C2","C3","C4","x0","gamma","HI_gal"]

# initial starting position
params = [4.5, 7.5, 0.0, 0.5, 3.1, 0.7, 3.23, 0.41, 4.59, 0.95, 20.5, 17.0]

Give the fitting parameters

In [ ]:
# emcee
nwalkers = 1000

Read in the star data

In [ ]:
# get the observed reddened star data
reddened_star = StarData(fstarname, path=f"{data_file_path}")
band_names = reddened_star.data["BAND"].get_band_names()
data_names = reddened_star.data.keys()

Plot the spectrum

In [ ]:
fig, ax = plt.subplots(figsize=(13, 10))
reddened_star.plot(ax)
ax.set_xscale("log")
ax.set_yscale("log")

Get the model data

In [ ]:
tlusty_models_fullpath = glob.glob(f"{model_file_path}/Models/wd_hubeny*.dat")
tlusty_models = [
    tfile[tfile.rfind("/") + 1 : len(tfile)] for tfile in tlusty_models_fullpath
]

# get the models with just the reddened star band data and spectra
modinfo = ModelData(
    tlusty_models,
    path=f"{model_file_path}/Models/",
    band_names=band_names,
    spectra_names=data_names,
)

Setup the fit parameters

In [ ]:
# parameter names
pnames = ["logT","logg", "logZ", "Av","Rv","C2","C3","C4","x0","gamma","HI_gal","HI_mw"]
pnames_fit = ["logT","logg", "Av","Rv","C2","C3","C4","x0","gamma","HI_gal"]

# initial starting position
params = [4.5, 7.5, 0.0, 0.5, 3.1, 0.7, 3.23, 0.41, 4.59, 0.95, 20.5, 17.0]

# min/max allowed values for each parameter
# some are based on the min/max of the stellar atmosphere grid

# not all these parameters are being fit
#   not fitting for met, log(HI) MW foreground
plimits = [
        [modinfo.temps_min, modinfo.temps_max],  # log(Teff)
        [modinfo.gravs_min, modinfo.gravs_max],  # log(g)
        [0.0, 0.0], # met
        [0.0, 1.0],   # Av
        [2.31, 5.59],   # Rv
        [-0.1, 5.0],  # C2
        [-1.0, 6.0],   # C3
        [-0.5, 1.5],   # C4
        [4.5, 4.9],   # xo
        [0.6, 1.5],   # gamma
        [18.0, 24.0], # log(HI) internal to galaxy
        [17.0, 22.0], # log(HI) MW foreground
    ]

# add Gaussian priors based on prior knowledge
#  sptype -> log(Teff), log(g)
#  galaxy metallicity -> log(Z)
ppriors = None
# ppriors = {}
#ppriors["logT"] = (params[0], 0.1)
#ppriors["logg"] = (3.1, 0.1)

In [ ]:
# plot initial guess
plot_data_model(reddened_star, modinfo, params)

Create the weight arrays based on the observed uncertainties

In [ ]:
# cropping info for weights
#  bad regions are defined as those were we know the models do not work
#  or the data is bad
ex_regions = [
    [8.23 - 0.1, 8.23 + 0.1],  # geocoronal line
    [8.7, 10.0],  # bad data from STIS
    [3.55, 3.6],
    [3.80, 3.90],
    [4.15, 4.3],
    [6.4, 6.6],
    [7.1, 7.3],
    [7.45, 7.55],
    [7.65, 7.75],
    [7.9, 7.95],
    [8.05, 8.1],
] / u.micron

weights = {}
for cspec in data_names:
    weights[cspec] = np.full(len(reddened_star.data[cspec].fluxes), 0.0)
    gvals = reddened_star.data[cspec].npts > 0

    weights[cspec][gvals] = 1.0 / reddened_star.data[cspec].uncs[gvals].value

    x = 1.0 / reddened_star.data[cspec].waves
    for cexreg in ex_regions:
        weights[cspec][np.logical_and(x >= cexreg[0], x <= cexreg[1])] = 0.0

# make the photometric bands have higher weight
#weights["BAND"] *= 10000.0

Package the fit info needed.  FitInfo class defines the likelihood functions as well.

In [ ]:
fitinfo = FitInfo(
    pnames,
    plimits,
    weights,
    parameter_priors=ppriors,
    stellar_velocity=velocity,
)

In [ ]:
# check that the initial starting position returns a valid value of lnprob and is within the parameter bounds
fitinfo.check_param_limits(params)
print(fitinfo.lnlike(params, reddened_star, modinfo))
print(fitinfo.lnprior(params))
print(fitinfo.lnprob(params, reddened_star, modinfo, fitinfo))

Useful function to plot data and model for specificed parameters

In [ ]:
# simple function to turn the log(likelihood) into the chisqr
#  requied as op.minimize function searchs for the minimum chisqr (not max likelihood like MCMC algorithms)
def nll(params, fixed_params, *args):
    full_params = get_full_params(params, fixed_params)
    return -fitinfo.lnprob(full_params, *args)

init_fit_params = get_fit_params(params)

# run the fit
result = op.minimize(
    nll, init_fit_params, method="Nelder-Mead", options={"maxiter": 10000}, 
    args=(fixed_params, reddened_star, modinfo, fitinfo)
)

# check the fit output
print(result["message"])

In [ ]:
# print results
fit_params = get_full_params(result["x"], fixed_params)
params_best = fit_params
pnames_extra = pnames

# print the best fit
for k, val in enumerate(params_best):
    print("{} # {}".format(val, pnames_extra[k]))


Plot the spectra with the best fit model

In [ ]:
# plot optimizer/minimizer best fit
plot_data_model(reddened_star, modinfo, fit_params)

Run emcee MCMC sampler to define uncertainties (bonus section)

In [ ]:
p0 = get_fit_params(fit_params)
ndim = len(p0)

def lnprob_mcmc(params, *args):
    full_params = get_full_params(params)
    return fitinfo.lnprob(full_params, *args)

nwalkers = 2 * ndim
nsteps = 100000

# setting up the walkers to start "near" the inital guess
p = [p0 * (1 + 0.01 * np.random.normal(0, 1.0, ndim)) for k in range(nwalkers)]

# setup the sampler
sampler = emcee.EnsembleSampler(
    nwalkers, ndim, lnprob_mcmc, args=(reddened_star, modinfo, fitinfo)
)

# do the full sampling
pos, prob, state = sampler.run_mcmc(p, nsteps, progress=True)

In [ ]:
# create the samples variable for later use
flat_samples = sampler.get_chain(discard=10000, flat=True)
print(flat_samples.shape)

# get the best fit values
# pnames_extra = pnames + ["E(B-V)", "N(HI)/A(V)", "N(HI)/E(B-V)"]
params_best = get_best_fit_params(sampler)
# fit_params = get_full_params(params_best)

def get_percentile_params(samples):
    """
    Determine the 50p plus/minus 33p vlaues
    """

    per_params = map(
        lambda v: (v[1], v[2] - v[1], v[1] - v[0]),
        zip(*np.percentile(samples, [16, 50, 84], axis=0)),
    )

    return per_params

# get the 16, 50, and 84 percentiles
params_per = get_percentile_params(flat_samples)

# save the best fit and p50 +/- uncs values to a file
# save as a single row table to provide a uniform format
#f = open(out_basename + "_fit_params.dat", "w")
#f.write("# best fit, p50, +unc, -unc\n")
print("best p50 -munc +unc # paramname")
for k, val in enumerate(params_per):
    print(f"{params_best[k]:.3f} {val[0]:.3f} {val[1]:.3f} {val[2]:.3f} # {pnames_fit[k]}")

In [ ]:
fit_params = get_full_params(params_best)
plot_data_model(reddened_star, modinfo, fit_params)

In [ ]:
nparam = len(params_best)
fig, axes = plt.subplots(nparam, figsize=(10, 15), sharex=True)
samples = sampler.get_chain()
labels = pnames_fit
for i in range(ndim):
    ax = axes[i]
    ax.plot(samples[:, :, i], "k", alpha=0.3)
    ax.set_xlim(0, len(samples))
    ax.set_ylabel(labels[i])
    ax.yaxis.set_label_coords(-0.1, 0.5)

axes[-1].set_xlabel("step number");

In [ ]:
import corner

fig = corner.corner(flat_samples, labels=labels)

In [ ]:
# usual gives an error
tau = sampler.get_autocorr_time()
print(tau)